# Importing Libraries

In [1]:
import numpy as np
import pandas as pd 

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk

stemmer = SnowballStemmer("english")


# Preprocessing

In [2]:
abbv_df = pd.read_csv('term_abb.csv')
abbv_df.head()

,terms,abbreviations
0,Auroracoin,AUR
1,BitConnect (inactive),BCC
2,Bitcoin Cash,BCH
3,Bitcoin,BTCorXBT
4,Dash,DASH


In [3]:
def_df = pd.read_csv('term_def.csv')
def_df.head()

,terms,definition1,definition2
0,51% attack,A hypothetical situation where more than half ...,NaN
1,51% attack protection,A protection mechanism implemented by several ...,NaN
2,AFK,Away From Keyboard; used on social media platf...,NaN
3,Airdrop,An event where a blockchain project distribute...,NaN
4,Altcoin,Any cryptocurrency that is an alternative to B...,NaN


In [4]:
def_df.isnull().sum()

terms            0
definition1      0
definition2    146
dtype: int64

In [5]:
len(def_df)

160

In [6]:
#Most of the definitions are null so droping the column
def_df.drop('definition2', axis=1, inplace = True)

In [7]:
# Joining the sentences to build a vocabulary

sentences = [''.join([def_df['terms'][i], def_df['definition1'][i]]) for i in range(len(def_df))]
sentences += [' '.join([abbv_df['terms'][i], abbv_df['abbreviations'][i]]) for i in range(len(abbv_df))]

In [8]:
sentences[:5]

['51% attackA hypothetical situation where more than half of the computing power on a blockchain network is controlled by one person or group, thus allowing them to dictate which transactions are verified. This would allow them to prevent other users from completing confirmed transactions and cause havoc within the system, and double-spend coins.',
 "51% attack protectionA protection mechanism implemented by several cryptocurrencies that require more than 50% of their total hashing power working together as one entity (which would make it difficult for attackers since they'd need even more resources and time) or if this threshold is below 100%, having an additional safeguard feature where at least 66% must agree with every transaction before sending, making them unable to double-spend anything without others noticing until these changes are made on the chain permanently.",
 "AFKAway From Keyboard; used on social media platforms like Twitter where users share their trading activity but 

In [9]:
# Preproceesing the senteces using lemmanization, removing stopwords and tokenizing it.

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [14]:
#Calling the function to clean the sentences
processed_sentences = []

for doc in sentences:
    processed_sentences.append(preprocess(doc))

In [15]:
# Create a dictionary from 'processed_sentences' containing the number of times a word appears 
# in the training set using gensim.corpora.Dictionary and call it 'dictionary'

dictionary = gensim.corpora.Dictionary(processed_sentences)

In [12]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 allow
1 attacka
2 blockchain
3 caus
4 coin
5 complet
6 comput
7 confirm
8 control
9 dictat
10 doubl


# Building corpus

In [16]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [17]:
document_num = 110
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 23 ("verifi") appears 1 time.
Word 42 ("requir") appears 1 time.
Word 62 ("messag") appears 1 time.
Word 282 ("digit") appears 2 time.
Word 320 ("mathemat") appears 1 time.
Word 442 ("alter") appears 1 time.
Word 447 ("document") appears 1 time.
Word 622 ("integr") appears 1 time.
Word 745 ("authent") appears 1 time.
Word 777 ("know") appears 1 time.
Word 840 ("satisfi") appears 1 time.
Word 841 ("scheme") appears 1 time.
Word 842 ("sender") appears 1 time.
Word 843 ("signaturea") appears 1 time.
Word 844 ("transit") appears 1 time.


# LDA Model


In [18]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [19]:
#For each topic exploring the words occuring in that topic and its relative weight


for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.025*"transact" + 0.021*"miner" + 0.018*"digit" + 0.018*"signatur" + 0.016*"block" + 0.013*"chain" + 0.013*"proof" + 0.012*"blockchain" + 0.012*"data" + 0.012*"requir"


Topic: 1 
Words: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan" + 0.009*"proof" + 0.009*"attack" + 0.009*"stake" + 0.009*"trade" + 0.008*"requir"


Topic: 2 
Words: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin" + 0.013*"block" + 0.011*"blockchain" + 0.011*"mine" + 0.011*"token" + 0.009*"network"


Topic: 3 
Words: 0.015*"access" + 0.015*"privat" + 0.012*"allow" + 0.012*"exchang" + 0.012*"euro" + 0.012*"item" + 0.009*"blockchain" + 0.009*"asset" + 0.009*"user" + 0.009*"bitcoin"


Topic: 4 
Words: 0.023*"block" + 0.022*"blockchain" + 0.019*"product" + 0.012*"term" + 0.012*"network" + 0.012*"money" + 0.009*"market" + 0.009*"valu" + 0.008*"currenc" + 0.008*"servic"


Topic: 5 
Words: 0.014*"distribut" + 0.011*"bitcoin" + 0.011*"time" + 0.0

# Tesing on test data

In [21]:
test_df=pd.read_csv('data.csv')
test_df.head()

,id,content
0,321712,Hey 👋 \n\nWe re using our bot:\n\nhttps://t.me...
1,321713,Good stuff \n\nI am surprised I took so long t...
2,321717,you are using a non-official one
3,321718,use the one that uniswap uses: https://thegrap...
4,321719,keep in mind this is a hot subgraph so it can ...


In [22]:
len(test_df)

44136

In [23]:
test_df.dropna(inplace=True)
test_df.isna().count()

id         43486
content    43486
dtype: int64

In [24]:
test_sentence = test_df['content']

In [25]:
for s in test_sentence:
    bow_vector = dictionary.doc2bow(preprocess(s))
    count=1
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        if score>0.65:
            print(index)
            print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
            print('--------------------------------------------------------------------')

1
Score: 0.6999240517616272	 Topic: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan"
--------------------------------------------------------------------
0
Score: 0.6834236979484558	 Topic: 0.025*"transact" + 0.021*"miner" + 0.018*"digit" + 0.018*"signatur" + 0.016*"block"
--------------------------------------------------------------------
2
Score: 0.6999248266220093	 Topic: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin"
--------------------------------------------------------------------
5
Score: 0.6999403238296509	 Topic: 0.014*"distribut" + 0.011*"bitcoin" + 0.011*"time" + 0.011*"applic" + 0.011*"transact"
--------------------------------------------------------------------
7
Score: 0.6999140977859497	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
5
Score: 0.6999807357788086	 Topic: 0.014*"distribut" + 0.01

7
Score: 0.6999614238739014	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
2
Score: 0.6999390125274658	 Topic: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin"
--------------------------------------------------------------------
8
Score: 0.6999654173851013	 Topic: 0.024*"token" + 0.023*"blockchain" + 0.019*"transact" + 0.018*"wallet" + 0.015*"coin"
--------------------------------------------------------------------
5
Score: 0.6999536752700806	 Topic: 0.014*"distribut" + 0.011*"bitcoin" + 0.011*"time" + 0.011*"applic" + 0.011*"transact"
--------------------------------------------------------------------
3
Score: 0.6999509334564209	 Topic: 0.015*"access" + 0.015*"privat" + 0.012*"allow" + 0.012*"exchang" + 0.012*"euro"
--------------------------------------------------------------------
1
Score: 0.6835769414901733	 Topic: 0.020*"transact" + 0.01

Score: 0.7749665975570679	 Topic: 0.014*"distribut" + 0.011*"bitcoin" + 0.011*"time" + 0.011*"applic" + 0.011*"transact"
--------------------------------------------------------------------
6
Score: 0.774952232837677	 Topic: 0.031*"coin" + 0.021*"investor" + 0.019*"price" + 0.018*"sell" + 0.017*"profit"
--------------------------------------------------------------------
0
Score: 0.6999762058258057	 Topic: 0.025*"transact" + 0.021*"miner" + 0.018*"digit" + 0.018*"signatur" + 0.016*"block"
--------------------------------------------------------------------
1
Score: 0.6999606490135193	 Topic: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan"
--------------------------------------------------------------------
6
Score: 0.8499706387519836	 Topic: 0.031*"coin" + 0.021*"investor" + 0.019*"price" + 0.018*"sell" + 0.017*"profit"
--------------------------------------------------------------------
6
Score: 0.6999468207359314	 Topic: 0.031*"coin" + 0.021*"investo

Score: 0.6667802333831787	 Topic: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan"
--------------------------------------------------------------------
5
Score: 0.6999409794807434	 Topic: 0.014*"distribut" + 0.011*"bitcoin" + 0.011*"time" + 0.011*"applic" + 0.011*"transact"
--------------------------------------------------------------------
0
Score: 0.6999779343605042	 Topic: 0.025*"transact" + 0.021*"miner" + 0.018*"digit" + 0.018*"signatur" + 0.016*"block"
--------------------------------------------------------------------
7
Score: 0.6999285817146301	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
0
Score: 0.7749649286270142	 Topic: 0.025*"transact" + 0.021*"miner" + 0.018*"digit" + 0.018*"signatur" + 0.016*"block"
--------------------------------------------------------------------
3
Score: 0.6999496817588806	 Topic: 0.015*"access" + 0.0

4
Score: 0.8499850630760193	 Topic: 0.023*"block" + 0.022*"blockchain" + 0.019*"product" + 0.012*"term" + 0.012*"network"
--------------------------------------------------------------------
7
Score: 0.6999410390853882	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
7
Score: 0.8199442028999329	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
5
Score: 0.8499637246131897	 Topic: 0.014*"distribut" + 0.011*"bitcoin" + 0.011*"time" + 0.011*"applic" + 0.011*"transact"
--------------------------------------------------------------------
2
Score: 0.7288974523544312	 Topic: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin"
--------------------------------------------------------------------
8
Score: 0.6999350786209106	 Topic: 0.024*"token" + 0.02

...

Score: 0.6999687552452087	 Topic: 0.031*"coin" + 0.021*"investor" + 0.019*"price" + 0.018*"sell" + 0.017*"profit"
--------------------------------------------------------------------
2
Score: 0.7749602198600769	 Topic: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin"
--------------------------------------------------------------------
7
Score: 0.7285429239273071	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
5
Score: 0.6999356150627136	 Topic: 0.014*"distribut" + 0.011*"bitcoin" + 0.011*"time" + 0.011*"applic" + 0.011*"transact"
--------------------------------------------------------------------
9
Score: 0.6999690532684326	 Topic: 0.023*"blockchain" + 0.019*"coin" + 0.012*"transact" + 0.012*"mine" + 0.012*"token"
--------------------------------------------------------------------
9
Score: 0.6999807357788086	 Topic: 0.023*"blockchain" + 0.019*"

2
Score: 0.7749594449996948	 Topic: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin"
--------------------------------------------------------------------
8
Score: 0.6999334096908569	 Topic: 0.024*"token" + 0.023*"blockchain" + 0.019*"transact" + 0.018*"wallet" + 0.015*"coin"
--------------------------------------------------------------------
0
Score: 0.84993976354599	 Topic: 0.025*"transact" + 0.021*"miner" + 0.018*"digit" + 0.018*"signatur" + 0.016*"block"
--------------------------------------------------------------------
7
Score: 0.8199734091758728	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
8
Score: 0.8199779391288757	 Topic: 0.024*"token" + 0.023*"blockchain" + 0.019*"transact" + 0.018*"wallet" + 0.015*"coin"
--------------------------------------------------------------------
1
Score: 0.7749716639518738	 Topic: 0.020*"transact" + 0.01

Score: 0.819974958896637	 Topic: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin"
--------------------------------------------------------------------
1
Score: 0.6999413371086121	 Topic: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan"
--------------------------------------------------------------------
7
Score: 0.6999749541282654	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
4
Score: 0.6999532580375671	 Topic: 0.023*"block" + 0.022*"blockchain" + 0.019*"product" + 0.012*"term" + 0.012*"network"
--------------------------------------------------------------------
1
Score: 0.6999414563179016	 Topic: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan"
--------------------------------------------------------------------
0
Score: 0.6999515295028687	 Topic: 0.025*"transact" + 0.021*"mine

2
Score: 0.6999533772468567	 Topic: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin"
--------------------------------------------------------------------
7
Score: 0.8499595522880554	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
6
Score: 0.6999598145484924	 Topic: 0.031*"coin" + 0.021*"investor" + 0.019*"price" + 0.018*"sell" + 0.017*"profit"
--------------------------------------------------------------------
1
Score: 0.6999789476394653	 Topic: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan"
--------------------------------------------------------------------
1
Score: 0.7374257445335388	 Topic: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan"
--------------------------------------------------------------------
7
Score: 0.6999659538269043	 Topic: 0.021*"hash" + 0.018*"blockchain"

Score: 0.699972927570343	 Topic: 0.023*"blockchain" + 0.019*"coin" + 0.012*"transact" + 0.012*"mine" + 0.012*"token"
--------------------------------------------------------------------
2
Score: 0.819956362247467	 Topic: 0.018*"currenc" + 0.018*"burn" + 0.016*"miner" + 0.013*"creat" + 0.013*"coin"
--------------------------------------------------------------------
8
Score: 0.6999436020851135	 Topic: 0.024*"token" + 0.023*"blockchain" + 0.019*"transact" + 0.018*"wallet" + 0.015*"coin"
--------------------------------------------------------------------
1
Score: 0.7749723196029663	 Topic: 0.020*"transact" + 0.014*"mine" + 0.009*"power" + 0.009*"improv" + 0.009*"mechan"
--------------------------------------------------------------------
7
Score: 0.696135938167572	 Topic: 0.021*"hash" + 0.018*"blockchain" + 0.014*"token" + 0.010*"coin" + 0.009*"transact"
--------------------------------------------------------------------
5
Score: 0.6833770275115967	 Topic: 0.014*"distribut" + 0.011*"bit